# Dashboard

This pipeline processes data into a form to drive the dashboard.

The parameters are injected using papermill.

In [ ]:
DEBUG = None
# DEBUG = 'live'
if DEBUG:
    import yaml
    with open('./params.yaml') as f:
        config = yaml.load(f, yaml.SafeLoader)['dashboards'][DEBUG]

    target = config['target']
    start = config['start']
    end = config['end']
    fixed_date = config['fixed_date']

In [ ]:
target: str
start: str
end: str
fixed_date: str

## Setup

Import some libraries

In [ ]:
import json
from datetime import date
from ast import literal_eval

import petl as etl
from utils.paths import PROCESSED, SITE
from utils.themes.programme_slice import ProgrammeSlice

Set the target directory, and ensure it exists

In [ ]:
TARGET = SITE / 'insights/dashboard' / target / '_data'
TARGET.mkdir(exist_ok=True, parents=True)

Calculate the date range. If the `fixed_date` parameter is `false`, make today the latest date.

In [ ]:
date_range = (
    date.fromisoformat(start),
    min(date.today(), date.fromisoformat(end)) if fixed_date == 'false' else date.fromisoformat(end),
    date.today(),
)

Write the timestamp file

In [ ]:
with open(TARGET / 'timestamp.json', 'w') as f:
    json.dump(dict(zip(['start', 'end', 'updated'], (str(x) for x in date_range))), f)

Load programme data

In [ ]:
programme_data = ProgrammeSlice(date_range=date_range[0:2])

## Events & projects data

In [ ]:
project_breakdown = (
    programme_data.events
    .melt(variables=[f for f in [
        'events', 'projected_events',
        'audience',
        'participants', 'participant_instances',
    ] if f in programme_data.events.header()])
    .selectnotnone('value')
    .aggregate(['project_name', 'evaluation_category', 'variable_group', 'variable'], sum, 'value')
    .recast()
)

In [ ]:
events = (
    project_breakdown
    .aggregate(['project_name', 'evaluation_category'], sum, 'events', field='events')
)

In [ ]:
project_summary = {
    'total': events.distinct('project_name').nrows(),
    'by_category': dict(events.aggregate('evaluation_category', len).records()),
    'names': sorted(list(events.cut('project_name').distinct().values('project_name'))),
}

In [ ]:
by_ticketed = (
    programme_data.events_data
    .selecteq('variable', 'events')
    .replace('ticketed', '', 'Unknown')
    .aggregate('ticketed', {
        'count': ('value', sum),
        'projects': ('project_name', set)
    })
    .convert('projects', lambda x: sorted(list(x)))
    .sort('count', reverse=True)
)

# free_events = (
#     programme_data.events_data
#     .selecteq('variable', 'events')
#     .search('ticketed', r'Yes \(Free:|^No$')
# )

# free_events.aggregate(['ticketed'], sum, 'value')

In [ ]:
events_summary = {
    'total': sum(events.values('events')),
    'free': by_ticketed.search('ticketed', r'Yes \(Free:|^No$').values('count').sum(),
    'by_category': dict(events.aggregate('evaluation_category', sum, 'events').records()),
    'by_project': {
        r[0]: {'category': r[1], 'count': r[2]}
        for r
        in events.records()
    },
    'by_ticketed': list(by_ticketed.dicts())
}

In [ ]:
with open(TARGET / 'events.json', 'w') as f:
    json.dump(
        events_summary,
        f,
        indent=2
    )

with open(TARGET / 'projects.json', 'w') as f:
    json.dump(
        project_summary,
        f,
        indent=2
    )

## Audiences

In [ ]:
audience_data = (
    project_breakdown
    .aggregate(['project_name', 'evaluation_category'], sum, 'audience', field='audience')
    .replace('audience', 0, None)
    .selectnotnone('audience')
)

In [ ]:
audience = {
    'total': sum(audience_data.values('audience')),
    'by_category': dict(
        audience_data
        .aggregate(['evaluation_category'], sum, 'audience')
        .records()
    ),
    'by_project': {
        r[0]: {'category': r[1], 'count': r[2]}
        for r in
        audience_data
        .aggregate('project_name', {
            'category': ('evaluation_category', lambda x: ', '.join(x)),
            'audience': ('audience', sum),
        })
        .records()
    },
}

In [ ]:
tickets_sold = (
    etl
    .fromcsv(PROCESSED / 'ticketing/tickets.csv')
    .selecteq('geography_type', 'oslaua')
    .rightjoin(
        etl
        .fromcsv(PROCESSED / 'ticketing/event-instances.csv')
        .convert('start', etl.dateparser('%Y-%m-%d %H:%M:%S'))
        .selectrangeopen('start', date_range[0], date_range[1])
        .selectcontains('eventType', 'MainProgrammeEvent')
        .cut('instance_id', 'event_name', 'event_id')
    )
    .cut('event_name', 'event_id', 'geography_code', 'count_of_tickets')
    .convertnumbers()
    .selectnotnone('count_of_tickets')
    .aggregate(['geography_code'], sum, 'count_of_tickets', field='count_of_tickets')
)


In [ ]:
ticketed_audience = {
    'sold': {
        'total': sum(tickets_sold.values('count_of_tickets')),
        'bradford': sum(tickets_sold.selecteq('geography_code', 'E08000032').values('count_of_tickets')),
    }
}

In [ ]:
with open(TARGET / 'audience.json', 'w') as f:
    json.dump(
        audience | { "ticketed": ticketed_audience },
        f,
        indent=2
    )

## Participants

### Programme

In [ ]:
programme_participants = (
    project_breakdown
    .selecteq('variable_group', 'community_programme')
    .cut(
        'project_name',
        'evaluation_category',
        'participants',
        'participant_instances'
    )
    .melt(['project_name', 'evaluation_category'])
    .selectnotnone('value')
    .aggregate(['project_name', 'evaluation_category', 'variable'], sum, 'value')
    .recast(samplesize=1_000_000)
    .selectgt('participants', 0)
)

In [ ]:
programme_participants

### Cultural learning

In [ ]:
cultural_learning_participants = (
    project_breakdown
    .selecteq('variable_group', 'cultural_learning')
    .cut(
        'project_name',
        'evaluation_category',
        'participants',
        'participant_instances'
    )
    .melt(['project_name', 'evaluation_category'])
    .selectnotnone('value')
    .aggregate(['project_name', 'evaluation_category', 'variable'], sum, 'value')
    .selectgt('value', 0)
    .recast(samplesize=1_000_000)
    .convertnumbers()
)

cultural_learning_settings = (
    etl
    .fromcsv(PROCESSED / 'programme/venues.csv')
    .selecteq('cultural_learning', 'True')
    .selectne('associated_project_count', '')
)

In [ ]:
cultural_learning_summary = (
    cultural_learning_participants
    .melt(variables=['participants', 'participant_instances'])
    .selectnotnone('value')
    .aggregate(['variable'], sum, 'value').data().dict()
) | { 
    'schools_engaged': cultural_learning_settings.nrows()
}

### Volunteers

In [ ]:
volunteer_signups = (
    etl
    .fromcsv(PROCESSED / 'volunteers/checkpoint-updates.csv')
    .convert('date', etl.dateparser('%Y-%m-%d'))
    .selectle('date', date_range[1])
    .convertnumbers(strict=False)
)

In [ ]:
# volunteer_shifts = (
#     etl
#     .fromcsv(PROCESSED / 'volunteers/shifts.csv')
#     .convert('date', etl.dateparser('%Y-%m-%d'))
#     .selecteq('type', 'BD25 Event')
#     .selectrangeopen('date', date_range[0], date_range[1])
#     .convertnumbers(strict=False)
# )

In [ ]:
volunteer_summary = {
    'total': sum(
        volunteer_signups
        .selectin('checkpoint',  ["1. Monitoring & Evaluation", "2. Sign Up to Induction", "3. Fully Inducted Volunteers"])
        .values('count')
    ),
    'people': {
        'checkpoints': dict(volunteer_signups.aggregate('checkpoint', sum, 'count').records()),
    },
    # 'events': {
    #     'count': volunteer_shifts.cut('rosterfy_event_name').distinct().nrows(),
    #     'names': list(volunteer_shifts.cut('rosterfy_event_name').distinct().values('rosterfy_event_name')),
    # },
    # 'shifts': {
    #     'count': volunteer_shifts.nrows(),
    #     'attended': sum(volunteer_shifts.values('attended')),
    #     'hours': sum(volunteer_shifts.values('hours')),
    # },
}

### Other figures

In [ ]:
other_figures = etl.fromcsv(PROCESSED / 'manual/manual-other-figures.csv').selecteq('dashboard key', target).convertnumbers()

In [ ]:
dict(other_figures.selecteq('variable', 'training_and_skills_development_participants').cut('source', 'value').rename('source', 'project_name').aggregate('project_name', sum, 'value', field='count').records())

In [ ]:
with open(TARGET / 'otherFigures.json', 'w') as f:
    json.dump(
        other_figures.aggregate('variable', sum, 'value').transpose().cutout('variable').dicts()[0],
        f,
        indent=2
    )

### Summary

Cultural Learning

In [ ]:
cl = (
     cultural_learning_participants
     .melt(variables=['participants', 'participant_instances'])
     .selectnotnone('value')
     .aggregate(['project_name', 'evaluation_category', 'variable'], sum, 'value')
     .recast()
     .replaceall(None, 0)
)

In [ ]:
def summarise_by_type(table):
    return {
        'count': table.values('count').sum(),
        'by_project': {
            r.project_name: { 'count': r['count'], 'category': r.get('evaluation_category', None) }
            for r in
            table.selectgt('count', 0).records()
        },
    }

by_type = {
    'community_participants': {
        'count': programme_participants.replaceall(None, 0).values('participants').sum(),
        'instances': programme_participants.replaceall(None, 0).values('participant_instances').sum(),
        'by_project': {
            r.project_name: {
                'count': r['count'],
                'instances': r['instances'],
                'category': r.get('evaluation_category')
            }
            for r in
            programme_participants.rename({
                'participants': 'count',
                'participant_instances': 'instances',
            }).records()
        },
    },
    'training_and_skills_development_participants': summarise_by_type(
        other_figures
        .selecteq('variable', 'training_and_skills_development_participants')
        .cut('source', 'value')
        .rename('source', 'project_name')
        .aggregate('project_name', sum, 'value', field='count')   
    ),
    'cultural_learning_participants': {
        'count': cl.values('participants').sum(),
        'by_project': {r['project_name']: dict(list(r.items())[1:]) for r in list(cl.dicts())},
    }
}

In [ ]:
by_type

In [ ]:
participant_summary = {
    'total': programme_participants.values('participants').sum()
        + cultural_learning_summary['participants']
        + volunteer_summary['total']
        + other_figures.selectcontains('variable', '_participants').values('value').sum(),

    'by_type': by_type,

    'cultural_learning': cultural_learning_summary,
    'volunteers': volunteer_summary,
}

In [ ]:
with open(TARGET / 'participants.json', 'w') as f:
    json.dump(
        participant_summary,
        f,
        indent=2
    )